In [37]:
import requests
import base64
import json
import re

In [39]:
def campos(fields):
    cad = ''
    for x in fields.keys():
        cad += x + ' = ' + fields[x] + ', '
    return cad[:-2]

def leeactzl(user, repo_name, path_to_file):
    json_url ='https://api.github.com/repos/{}/{}/contents/{}'.format(user, repo_name,
                                                                      path_to_file)
    response = requests.get(json_url) #get data from json file located at specified URL 

    if response.status_code == requests.codes.ok:
        jsonResponse = response.json()  # the response is a JSON
        #the JSON is encoded in base 64, hence decode it
        content = base64.b64decode(jsonResponse['content'])
        #convert the byte stream to string
        jsonString = content.decode('utf-8')
        return json.loads(jsonString)
 
    else:
        return 'Content was not found.'
    
def calcquerys(finalJson):

    for registro in finalJson:
        if finalJson[0]['firma'] == hashlib.sha1(registro["instruccion"].encode('utf-8')).hexdigest():
            dt_query = json.loads(registro['instruccion'])
            querys = []

            for y in dt_query.keys():
                if dt_query[y]['op'] == 'UPDATE':
                    query  = str(dt_query[y]['op']) + ' ' + y + ' SET ' + campos(dt_query[y]['fields'])
                    query += ' WHERE '
                elif dt_query[y]['op'] == 'INSERT INTO':
                    query  = query = str(dt_query[y]['op']) + ' ' + y
                    query += ' ' + str(tuple(list(dt_query[y]['fields'].keys())))+ ' VALUES '
                    query +=  str(tuple(list(dt_query[y]['fields'].values())))
                    query  = query.replace("'", "")
                    filtro_update = 'coduser = ' + dt_query[y]['fields']['coduser']

                querys.append(query)

            break # <--== Pilas quitar en produccion

    querys = [query + filtro_update if 'UPDATE' in query else query for query in querys]

    patron = re.compile('[0-9]{4}-[0-9]{2}-[0-9]{2}')

    for i, query in enumerate(querys):
        for r in patron.findall(querys[i]):
            querys[i] = querys[i].replace(r, "'" + r + "'")

    return querys

In [ ]:
user = 'sistelca'
repo_name = 'desechosSolidos'
path_to_file = 'data_actzl.json'

finalJson = leeactzl(user, repo_name, path_to_file)


In [40]:
# Ojo hay que registrar el hash en db receptora para vefiricar que no este y ejecutar 
# query

print(calcquerys(finalJson))


["UPDATE datos_red SET fech_pag = '2021-03-21' WHERE coduser = 347", "INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc) VALUES (347, '2021-04-05', 100000, '2021-04-21')", "UPDATE datos_per SET fech_ing = '2021-04-05', fech_ven = '2021-04-21' WHERE coduser = 347"]
